In [1]:
# import statements

import pandas as pd

In [2]:
# Reading csv files and storing

data_source1 = pd.read_csv("bollywood.csv")
data_source2 = pd.read_csv("bollywood - 2.csv")

In [3]:
# dropping the ID column since it is same as index

data_source1.drop(['No'], axis=1, inplace=True)
data_source2.drop(['Movie_ID'], axis=1, inplace=True)

In [4]:
# Changing the column names to match in both files

data_source1=data_source1.rename(columns={'RelDate': 'Release_Date',
                                          'MovieName': 'Movie_Name',
                                          'ReleaseTime': 'Release_Time',
                                          'YoutubeViews': 'Youtube_Views',
                                          'YoutubeLikes': 'Youtube_Likes',
                                          'YoutubeDislikes': 'Youtube_Dislikes',
                                          'Budget': 'Budget_in_Crore',
                                          'BoxOfficeCollection': 'BoxOfficeCollection_in_Crore'})
data_source2=data_source2.rename(columns={'BudgetCr': 'Budget_in_Crore',
                                          'BoxOfficeCollectionCr': 'BoxOfficeCollection_in_Crore'})

In [5]:
print("Initial Shape")
print(data_source1.shape)
print(data_source2.shape)

Initial Shape
(51, 10)
(99, 9)


### Q1. Identify and rectify the eight prominent data quality issues present in the given datasets.

In [6]:
# Method to remove duplicate values

def duplicate_removal(df):
    duplicate_rows = df[df.duplicated()]
    if not duplicate_rows.empty:
        df.drop_duplicates(inplace=True)
    print("Shape After remove of duplicates {}".format(df.shape))
    return df

In [7]:
#method to check for null values and fill the values accordingly

def null_check(df):
    if df.isnull().sum().sum() != 0:
        print(df.isnull().sum())
        index_list = df[df['Youtube_Likes'].isnull()].index.tolist()
        for index in index_list:
            new_value = df.loc[index, 'YoutubeLikeDislikes'] - df.loc[index, 'Youtube_Dislikes']
            df.loc[index, 'Youtube_Likes'] = new_value
        index_list = df[df['Youtube_Dislikes'].isnull()].index.tolist()
        for index in index_list:
            new_value = df.loc[index, 'YoutubeLikeDislikes'] - df.loc[index, 'Youtube_Likes']
            df.loc[index, 'Youtube_Dislikes'] = new_value
        return df
    else:
        print("No null values found")
        return df

In [8]:
# method to remove unwanted substrings
def substring_removal(df, sub):
    if sub == "cr":
        df["Budget_in_Crore"] = df["Budget_in_Crore"].str.replace(sub, "", case=False)
        df["BoxOfficeCollection_in_Crore"] = df["BoxOfficeCollection_in_Crore"].str.replace(sub, "", case=False)
    if sub == "Genre":
        df["Genre"] = df["Genre"].str.replace(sub, "", case=False)
    return df

In [9]:
# Converting the data types to the right format
def datatype_conversion(df):
    df['Budget_in_Crore'] = pd.to_numeric(df['Budget_in_Crore'])
    df['Genre'] = df['Genre'].astype(str)
    df['BoxOfficeCollection_in_Crore'] = pd.to_numeric(df['BoxOfficeCollection_in_Crore'])
    try:
        df['Release_Date'] = pd.to_datetime(df['Release_Date'], errors="raise", format='%d-%b-%Y')
    except:
        df['Release_Date'] = pd.to_datetime(df['Release_Date'], errors="raise", infer_datetime_format=True)
    return df

In [10]:
# performing Release time mapping to sync both files

def data_mapping(df):
    mapping = {'FS': 1, 'HS': 2, 'LW': 3, 'N': 4}
    df.Release_Time = [mapping[item] for item in df.Release_Time]
    return df

In [11]:
# Removing duplicate rows if any
data_source1 = duplicate_removal(data_source1)
data_source2 = duplicate_removal(data_source2)

Shape After remove of duplicates (51, 10)
Shape After remove of duplicates (99, 9)


In [12]:
# Performing null check and providing values
data_source1 = null_check(data_source1)
data_source1 = null_check(data_source1)

data_source2 = null_check(data_source2)

Release_Date                    0
Movie_Name                      0
Release_Time                    0
Genre                           0
Budget_in_Crore                 0
BoxOfficeCollection_in_Crore    0
Youtube_Views                   0
Youtube_Likes                   1
Youtube_Dislikes                3
YoutubeLikeDislikes             0
dtype: int64
No null values found
No null values found


In [13]:
# dropping the odd out column after used to fill the null values in YoutubeLike & YoutubeDislikes field

data_source1.drop(['YoutubeLikeDislikes'], axis=1, inplace=True)

In [14]:
# performing substring removal
data_source1 = substring_removal(data_source1, "cr")
data_source1 = substring_removal(data_source1, "Genre")

In [15]:
# correcting the typo mistakes.
data_source1['Release_Date'] = data_source1['Release_Date'].replace('Sqp-19-2014', 'Sep-19-2014')
data_source1['Release_Date'] = data_source1['Release_Date'].replace('Jum-27-2014', 'Jun-27-2014')
data_source1['Release_Date'] = data_source1['Release_Date'].replace('juu-6-2014', 'jun-6-2014')
data_source1['Release_Date'] = data_source1['Release_Date'].replace('Feb-21-204', 'Feb-21-2014')
data_source1["Release_Time"] = data_source1["Release_Time"].replace(44,4)

In [16]:
# performing data mapping for release time column in file 2
data_source2 = data_mapping(data_source2)

In [17]:
data_source1 = datatype_conversion(data_source1)
data_source2 = datatype_conversion(data_source2)

In [18]:
print("Final Shape")
print(data_source1.shape)
print(data_source2.shape)

Final Shape
(51, 9)
(99, 9)


In [19]:
# Merging 2 dataframes
data_source = pd.concat([data_source1, data_source2], ignore_index=True)
print("Shape after mergeing both data {}".format(data_source.shape))

Shape after mergeing both data (150, 9)


In [20]:
# Dropping duplicates w.r.t movie name post merge
data_source = data_source.drop_duplicates(subset='Movie_Name', keep="first")
data_source['Genre'] = data_source['Genre'].str.strip()
print("Shape after removing duplicates post merge {}".format(data_source.shape))

Shape after removing duplicates post merge (149, 9)


### Q2. How many records are present in the dataset? Print the metadata information of dataset?

In [21]:
print(data_source.info(verbose=True, null_counts=False))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 149
Data columns (total 9 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   Release_Date                  datetime64[ns]
 1   Movie_Name                    object        
 2   Release_Time                  int64         
 3   Genre                         object        
 4   Budget_in_Crore               int64         
 5   BoxOfficeCollection_in_Crore  float64       
 6   Youtube_Views                 int64         
 7   Youtube_Likes                 float64       
 8   Youtube_Dislikes              float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 11.6+ KB
None


### Q3. How many movies in each genre got released in different release times?

In [22]:
data_source.groupby(['Release_Time','Genre',]).size().to_frame('Movie_Counts')

Movie_Counts
Release_Time Genre                 
1            Action               3
             Comedy               3
             Drama                4
             Romance              3
             Thriller             4
2            Action               3
             Comedy               5
             Drama                6
             Romance              3
             Thriller             1
3            Action               3
             Comedy               5
             Drama                1
             Romance              4
             Thriller             2
4            Action              15
             Comedy              23
             Drama               24
             Romance             15
             Thriller            22

### Q4. Which month of the year, maximum movie releases are seen?

In [38]:
p_data = data_source.groupby([data_source.Release_Date.dt.year, data_source.Release_Date.dt.month]).size().to_frame('Movie_Counts')
p_data.groupby(['Release_Date'], sort=False)['Movie_Counts'].max().to_frame('Movie_Counts')
# data_source['date_year'] = data_source.groupby([data_source.Release_Date.dt.year]).size()
# data_source['date_month'] = data_source.groupby([data_source.Release_Date.dt.month]).size()

# data_source.groupby(['date_year','date_month']).size().count()

# data_source.groupby([pd.DatetimeIndex(data_source['Release_Date']).year, pd.DatetimeIndex(data_source['Release_Date']).month]).size().to_frame('Movie_Counts')
# date_groupby.groupby(['Release_Date']).agg({'Movie_Counts': max})


# date_groupby = data_source.groupby([data_source.Release_Date.dt.year, data_source.Release_Date.dt.month]).size().to_frame('Movie_Counts')
# date_groupby.groupby(['Release_Date']).agg({'Movie_Counts': max})

# data_source.groupby([data_source.Release_Date.dt.month]).size()


# data_source.groupby([data_source.Release_Date.dt.year, data_source.Release_Date.dt.month]).size().to_frame('Movie_Counts')
# data_source.groupby([data_source.Release_Date.dt.year])['M'].transform(max)
# data_source.groupby([data_source.Release_Date.dt.year, data_source.Release_Date.dt.month]).size().to_frame('Movie_Counts')["Movie_Counts"].agg(["max", "min"])

# data_source.groupby(data_source.Release_Date.dt.year)['']
# data_source.groupby(data_source.Release_Date.dt.year)[''].agg(['sum','count'])
# data_source.groupby([data_source['Release_Date'].dt.year, data_source['Release_Date'].dt.month]).agg({'count'})

,Movie_Counts
Release_Date,
2013,9
2014,12
2015,5


### Q5. Which month of the year typically sees most releases of high budgeted movies, that is, movies with budget of 25 crore or more? 

In [45]:
budget_source = data_source.query('Budget_in_Crore >= 25', inplace =False)
temp = budget_source.groupby([data_source.Release_Date.dt.year, data_source.Release_Date.dt.month]).size().to_frame('Movie_Counts > 25Cr')
month = temp.groupby(['Release_Date'], sort=False)['Movie_Counts > 25Cr'].transform(max) == temp['Movie_Counts > 25Cr']
month.loc[month]


# temp.groupby(['Release_Date'], sort=False)['Movie_Counts > 25Cr'].max().to_frame('Movie_Counts > 25Cr')
# df.groupby(['Mt'])['count'].transform(max) == df['count']

Release_Date  Release_Date
2013          7               True
2014          2               True
              3               True
              4               True
2015          1               True
Name: Movie_Counts > 25Cr, dtype: bool

### Q6. Which are the top 10 movies with maximum return of investment (ROI)? Calculate ROI as (Box office collection – Budget) / Budget?

In [26]:
data_source['ROI'] = (data_source['BoxOfficeCollection_in_Crore'] - data_source['Budget_in_Crore']) / data_source['Budget_in_Crore']
data_source.nlargest(10, 'ROI').reset_index()[['Movie_Name', 'ROI']]

,Movie_Name,ROI
0,Rajdhani Express,53.714286
1,Aashiqui 2,8.166667
2,PK,7.647059
3,Grand Masti,7.514286
4,The Lunchbox,7.500000
5,Fukrey,6.240000
6,Mary Kom,5.933333
7,Shahid,5.666667
8,Humpty Sharma Ki Dulhania,5.500000
9,Bhaag Milkha Bhaag,4.466667
